## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df.copy().drop(columns=['EIN','NAME'])
application_df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df2['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace= application_df2['APPLICATION_TYPE'].value_counts()
application_types_to_replace= application_types_to_replace[application_types_to_replace < 1000]
application_types_to_replace=application_types_to_replace.copy()

# Replace in dataframe
def update_app(value):
    if value in application_types_to_replace: #['T9','T13','T12','T2','T25','T14','T29','T15','T17']:
        return 'OTHER'
    else:
        return value
application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].apply(update_app)
application_df2['APPLICATION_TYPE'].value_counts()

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
OTHER     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_to_replace = application_df2['CLASSIFICATION'].value_counts().copy()
classifications_to_replace[classifications_to_replace > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classifications_to_replace[classifications_to_replace < 1000]
classifications_to_replace = classifications_to_replace.copy()

# Replace in dataframe
def update_app(value):
    if value in classifications_to_replace:
        return 'OTHER'
    else:
        return value
application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].apply(update_app)
application_df2['CLASSIFICATION'].value_counts()

# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
OTHER     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
numerical = ['STATUS','IS_SUCCESSFUL']
to_convert = [
    'APPLICATION_TYPE',
    'AFFILIATION',
    'CLASSIFICATION',
    'USE_CASE',
    'ORGANIZATION',
    'INCOME_AMT',
    'SPECIAL_CONSIDERATIONS',
    'ASK_AMT'
]

# Apply pd.get_dummies() on the specified columns
dummies = pd.get_dummies(application_df2[to_convert])

# Concatenate the dummy variables with the original DataFrame
application_df2 = pd.concat([application_df2, dummies], axis=1)
application_df2 = application_df2.drop(columns=to_convert)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df2.drop(columns='IS_SUCCESSFUL')
y = application_df2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
xs = scaler.fit(X_train)

# Scale the data
X_train_scaled = xs.transform(X_train)
X_test_scaled = xs.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=39, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4000      
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 24,301
Trainable params: 24,301
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [17]:
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 4ms/step - loss: 0.5702 - accuracy: 0.7237
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5571 - accuracy: 0.7318
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5538 - accuracy: 0.7325
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5535 - accuracy: 0.7329
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5520 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5506 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7357
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7353
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5485 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5849 - accuracy: 0.7259 - 663ms/epoch - 2ms/step
Loss: 0.5849487781524658, Accuracy: 0.7259474992752075


In [19]:
# Export our model to HDF5 file
import h5py
hdf5_filename = 'AlphabetSoupCharity_Optimization.h5'
nn.save(hdf5_filename)
print('Model saved to : ',hdf5_filename)

Model saved to :  AlphabetSoupCharity_Optimization.h5
